In [1]:
# Import packages
import keras
import pandas as pd 
import numpy as np 
from tqdm import tqdm
from keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from keras.models import load_model
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras import models
from keras import layers
from keras import optimizers
import sklearn
from sklearn.model_selection import train_test_split
from statistics import mean
import unidecode

In [2]:
def arrange_data(df):
    
    image_data = []
    img_paths = np.asarray(df.iloc[:, 0]) #First column is the image paths
    
    for i in tqdm(range(len(img_paths))):
    
        img = tf.keras.utils.load_img(img_paths[i],target_size=(200,150,3))
        img = tf.keras.utils.img_to_array(img)
        img = img/255
        image_data.append(img)
        
        
    X = np.array(image_data)
    Y = np.array(df.iloc[:,1:7])
    
    return X, Y

In [3]:
# Load the model in
model = load_model('Model_4d.h5')

In [4]:
test_df = pd.read_csv('testingData.csv')

test_df['Image Path'] = test_df['Image Path'].str.replace('/Users/johnhope/Desktop/GitHub/AlbumArtCoverColors/data/','')
test_df['Image Path'] = test_df['Image Path'].str.replace('/images','')

X_test, Y_test = arrange_data(test_df)

100%|██████████| 637/637 [00:11<00:00, 55.81it/s]


In [5]:
# Make predictions
pred = model.predict(np.array(X_test))

# Choose highest probability as predicted genre
pred_genre = np.zeros_like(pred)
pred_genre[np.arange(len(pred)), pred.argmax(1)] = 1
pred_genre=pred_genre.astype(int)

20/20 [==============================] - 81s 4s/step


In [6]:
count = 0
for i in range(0,len(pred_genre)-1):
        if list(pred_genre[i]) == list(Y_test[i]):
            count += 1
        
count/len(pred_genre) # accuracy

0.2794348508634223

In [8]:
pred_df = pd.DataFrame(pred_genre, columns = ['Pred Pop','Pred Hip Hop', 'Pred Country', 'Pred Jazz', 'Pred Rock', 'Pred Alternative'])

full_df = test_df.join(pred_df)

In [9]:
full_df

,Image Path,Pop,Hip Hop,Country,Jazz,Rock,Alternative,Pred Pop,Pred Hip Hop,Pred Country,Pred Jazz,Pred Rock,Pred Alternative
0,pop/Wreck.jpg,1,0,0,0,0,0,0,0,1,0,0,0
1,pop/War.jpg,1,0,0,0,0,0,0,0,0,0,1,0
2,pop/Sensational.jpg,1,0,0,0,0,0,0,0,0,0,1,0
3,pop/<32.jpg,1,0,0,0,0,0,0,0,0,0,1,0
4,pop/IceePop.jpg,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,alternative/GhostsoftheGreatHighway.jpg,0,0,0,0,0,1,0,0,1,0,0,0
633,"alternative/ToSeetheFuture(From""SwordArtOnline...",0,0,0,0,0,1,0,0,0,1,0,0
634,alternative/LovinYou.jpg,0,0,0,0,0,1,1,0,0,0,0,0
635,alternative/AlternativeThoughts.jpg,0,0,0,0,0,1,0,0,0,1,0,0


In [15]:
pop_acc = full_df[full_df['Pop']==1]['Pred Pop'].sum()/len(full_df[full_df['Pop']==1])
hiphop_acc = full_df[full_df['Hip Hop']==1]['Pred Hip Hop'].sum()/len(full_df[full_df['Hip Hop']==1])
country_acc = full_df[full_df['Country']==1]['Pred Country'].sum()/len(full_df[full_df['Country']==1])
jazz_acc = full_df[full_df['Jazz']==1]['Pred Jazz'].sum()/len(full_df[full_df['Jazz']==1])
rock_acc = full_df[full_df['Rock']==1]['Pred Rock'].sum()/len(full_df[full_df['Rock']==1])
alt_acc = full_df[full_df['Alternative']==1]['Pred Alternative'].sum()/len(full_df[full_df['Alternative']==1])

In [17]:
print(pop_acc, hiphop_acc, country_acc, jazz_acc, rock_acc, alt_acc)

0.1188118811881188 0.3177570093457944 0.2916666666666667 0.125 0.6170212765957447 0.052083333333333336
